In [ ]:
import pickle
import pandas as pd

# Read the source file of raw commit data
pred_commits = pd.read_csv('C:/Users/aveli/Downloads/elasticray.csv')

In [ ]:
# Prepare the features from raw commit data

pred_commits['total_changed'] = pred_commits['lines_added']+pred_commits['lines_removed']
pred_commits['size'].loc[pred_commits['size'] == 0] = pred_commits['total_changed']
pred_commits['ratio_changed'] = pred_commits['total_changed']/pred_commits['size']
pred_commits['rated_complexity'] = pred_commits['ratio_changed']*pred_commits['complexity']
pred_commits['total_dmm_size'] = pred_commits['total_changed']*pred_commits['dmm_unit_size']
pred_commits['total_dmm_unit_complexity'] = pred_commits['total_changed']*pred_commits['dmm_unit_complexity']
pred_commits['total_dmm_unit_interfacing'] = pred_commits['total_changed']*pred_commits['dmm_unit_interfacing']

# We picked the sqrt of no_of_mod_files to reduce weightage of this feature
pred_commits['scaled_rated_complexity'] = pred_commits['rated_complexity'] * (pred_commits['no._of_mod_files'] ** 0.5)


pred_ml_commits = pred_commits[['hash','Author','Committer','committed_date','total_changed','rated_complexity',
                               'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity']]

# Resetting the frame's index. It is required to retain the integrity of the frame
pred_ml_commits = pred_ml_commits.reset_index().drop(columns = 'index')

# Author column needs to be dropped before converting the all the fields into numeric types
pred_ml_commits_na = pred_ml_commits.drop(columns = ['Author','hash','Committer','committed_date'])

# Converting the fields to numeric types, filling the NaNs with zeros
pred_ml_commits_numeric = pred_ml_commits_na.apply(pd.to_numeric,errors ='coerce').fillna(0)

# Adding teh Author column back
pred_ml_commits_numeric['Author'] = pred_ml_commits['Author']
pred_ml_commits_numeric['hash'] = pred_ml_commits['hash']
pred_ml_commits_numeric['Committer'] = pred_ml_commits['Committer']
pred_ml_commits_numeric['committed_date'] = pred_ml_commits['committed_date']
pred_ml_commits_numeric_na = pred_ml_commits_numeric.drop(columns=['Author','hash','committed_date','Committer'])

In [ ]:
# Get the pickled model
filename = 'C:/Users/aveli/Downloads/finalized_model.sav'
xboost_model = pickle.load(open(filename, 'rb'))

In [ ]:
# Predict the cluster using the model

predicted_clusters = xboost_model.predict(pred_ml_commits_numeric_na)
pred_ml_commits_numeric['predicted_cluster'] = predicted_clusters

In [ ]:
pred_ml_commits_numeric.to_csv('C:/Users/aveli/Downloads/elasticray_predicted.csv')